In [1]:
from ads_query_eval.config import get_terminus_client
from ads_query_eval.app.bootstrap import QUERIES

client = get_terminus_client()

In [2]:
client.replace_document([
    {
        "@type": "Query",
        "query_literal": 'full:"coronal mass ejection"',
        "@capture": "id_query_cme"
    },
    {
        "@type": "Retrieval",
        "query": {"@ref": "id_query_cme"},
        "status": "completed",
        "done": True,
        "@capture": "id_retrieval_cme"
    },
    {
        "@type": "RetrievedItemsList",
        "retrieval": {"@ref": "id_retrieval_cme"},
        "@capture": "id_retrieved_items_list_cme"
    },
    {
        "retrieved_items_list": {"@ref": "id_retrieved_items_list_cme"},
        "retrievable_item": {"@ref": "id_item_2022npjAM..14...43F"}
    },
    {
        "ads_bibcode": "2022npjAM..14...43F",
        "@capture": "id_item_2022npjAM..14...43F"
    },
    {
        "fqn": "ads_query_eval.frame.evaluators.hal",
        "version": "0.1",
        "config": {"mode": "awesome"},
        "@capture": "id_evaluator_hal"
    },
    {
        "evaluator": {"@ref": "id_evaluator_hal"},
        "p_at_25": 0.55,
        "r_at_1000": 0.10,
        "done": True,
        "status": "completed",
        "@capture": "id_evaluation_hal"
    },
    {
        "category": "SensedEvaluationOfRetrievedItemsByProcedure",
        "operations": [{"@ref": "id_evaluation_hal"}]
    },
    {
        "category": "ScheduledRetrieval",
        "operations": [{"@ref": "id_retrieval_cme"}]
    }
], commit_msg="Adding test docs", create=True)


list(client.get_all_documents())

[{'@id': 'RetrievedItem/4071087f3acb9efadb40a2714174960bd3969669aebae6c9bc414322e6481f33',
  '@type': 'RetrievedItem',
  'retrievable_item': 'RetrievableItem/2022npjAM..14...43F',
  'retrieved_items_list': 'RetrievedItemsList/9d6fa694063f7919d8cab449ea8bf091c9a23a986ed65ec7abc381e840a7f629'},
 {'@id': 'RetrievedItem/1be6b3978b969838a22f3ae6319b676f357c249bf8a5df4414aee4d688d6e7a6',
  '@type': 'RetrievedItem',
  'retrievable_item': 'RetrievableItem/2022npjAM..14...43F',
  'retrieved_items_list': 'RetrievedItemsList/7331b2004b3760ee0c735a22025d8f1681454a3424cf5151a0d45477089a6e6e'},
 {'@id': 'JobRun/af8d0b2cc5161f6cea472aeeb1578f942076086bd5e979e07caa9dbc4f2fb201',
  '@type': 'JobRun',
  'category': 'SensedEvaluationOfRetrievedItemsByProcedure',
  'operations': ['Evaluation/585795f1e1115c3d37b3673ae48a1e08653906852c8f751899a4e2fcdb0900b1']},
 {'@id': 'JobRun/c9bafaa8599c0e1d182e977c86f1d04378fe47332823780b5b8cc80c5f9000f4',
  '@type': 'JobRun',
  'category': 'ScheduledRetrieval',
  'oper

In [3]:
import requests

def tget(url):
    client._check_connection()
    return requests.get(
        url,
        headers=client._default_headers,
        auth=client._auth(),
    )

def tget_ttl(dbid, which=None):
    assert which in {"schema","instance"}
    rv = tget(f"http://terminus:6363/api/triples/admin/{dbid}/local/branch/main/{which}?format=turtle")
    for line in rv.json().split('\n'):
        print(line)
    return rv

tget_ttl(client.db, which="schema")
tget_ttl(client.db, which="instance")

@base <terminusdb:///schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix woql: <http://terminusdb.com/schema/woql#> .
@prefix json: <http://terminusdb.com/schema/json#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix xdd: <http://terminusdb.com/schema/xdd#> .
@prefix vio: <http://terminusdb.com/schema/vio#> .
@prefix sys: <http://terminusdb.com/schema/sys#> .
@prefix api: <http://terminusdb.com/schema/api#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix doc: <data/> .

doc:Cons\/12e85906ce2857d76a96c89deb85124da7a83bc2f3d984dc36400040f4507c70
  a rdf:List ;
  rdf:first <schema#JobCategory/SensedEvaluationOfRetrievedItemsByProcedure> ;
  rdf:rest rdf:nil .

doc:Cons\/172c75c76abb349e0614152490746d0f5a5c94f4b7f94a41beaf9703db956f5b
  a rdf:List ;
  rdf:first <schema#JobCategory/ScheduledRetrieval> ;
  rdf:rest doc:Cons\/12e85906ce2857d76a96c89deb85124da7a83bc2f3d984dc36400040f4507c

<Response [200]>